### Oppstart

Sjekk hvilken chrome du bruker under help

Last ned chromedriver
https://sites.google.com/a/chromium.org/chromedriver/downloads
Legg på /Users/User/Documents/WebDriver

Husk å sette path til chromedriver.
1. Open up Terminal.
1. Run sudo nano /etc/paths.
1. Enter your password.
1. Go to the bottom of the file and enter the path you wish to add.
1. My PATH looks like: /usr/local/bin
1. Control-x to quit.
1. Y to save.
1. Press enter to confirm

In [ ]:
from bs4 import BeautifulSoup
import requests as re
import csv
from selenium import webdriver
import ipywidgets as widgets
import time
import os

In [ ]:
bruker = widgets.Text(
    value='Brukernavn',
    placeholder='Mailadresse hos nets',
    description='Mail:',
    disabled=False
)
passwrd = widgets.Text(
    value='Passord',
    placeholder='Passord hos nets',
    description='Passord:',
    disabled=False
)
display(bruker, passwrd)

In [ ]:
print(bruker.value)
print(passwrd.value)

In [ ]:
root = "Documents/Nets_Scrape/"
output_csv = "1desember-4jan_side1.csv"

In [ ]:
#local_file = "Easyadministration.html"
url = 'https://portal.dibspayment.eu/payouts'
#page = re.get(URL)

In [ ]:
if not os.path.exists(root):
    os.makedirs(root)

In [ ]:
chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(executable_path=chrome_path)
driver.get(url)

In [ ]:
# Login om nødvendig
driver.find_element_by_id('loginEmail').send_keys(bruker.value)
driver.find_element_by_id('loginPassword').send_keys(passwrd.value)
driver.find_element_by_css_selector('button#login').click()

In [ ]:
#driver.get(url)

In [ ]:
##################################################################
# START OVER FROM HERE AFTER NAVIGATING TO WANTED DATE RANGE #
##################################################################

soup = driver.page_source

In [ ]:
soup = BeautifulSoup(soup, 'html.parser')
main_trs = soup.find('table', class_='table-striped').find_all('tr')

In [ ]:
def per_subpage(soup):
    main_trs = soup.find('table', class_='table-striped').find_all('tr')
    
    header = []
    main_list = []
    count = 0

    hr_cells = main_trs[0].find_all('th')
    for cell in hr_cells:
        header.append(cell.text.strip().replace("\r", "").replace("\n", "").replace('NOK', ''))

    for row in main_trs:
        under_main_list = []
        cells = row.find_all('td')
        if cells:
            for cell in cells:
                under_main_list.append(cell.text.strip().replace("\r", "").replace("\n", ""))
            main_list.append(under_main_list)
            
    main_temp = [x for x in main_list[:-1]]

    main_trans = soup.find('div', class_='block-hero')

    main_trans_head = []
    main_trans_cont=  []

    referanse = soup.find('span', class_='payout-referene').text.strip()
    main_trans_head.append(referanse.split(": ")[0])
    main_trans_cont.append(referanse.split(": ")[1])

    main_trans_head.append(main_trans.find('p', class_='payout-date').text.strip().split("\n")[0])
    main_trans_cont.append(main_trans.find('span', class_='date-cell').text.strip())

    mntrns_maintb = main_trans.find('table', class_='order-summary-list').find_all('tr')
    count = 0
    for row in mntrns_maintb:
        cells = row.find_all('td')
        for cell in cells:
            if count % 2 == 0:
                main_trans_head.append(cell.text.strip().replace("\r", "").replace("\n", ""))
            else:
                cont = cell.text.strip().replace("\r", "").replace("\n", "")
                if "NOK" in cont:
                    cont = cont.split("NOK")[0]
                main_trans_cont.append(cont)
            count += 1

    results = soup.find('table', class_='table-striped')

    header_elements = results.find_all(class_='cell-title')

    headers = []
    for elem in header_elements:
        headers.append(elem.text.strip())

    rows_all = results.find_all('tr', class_='ng-star-inserted')

    rows = []

    for rows_elem in rows_all:
        row_cell = rows_elem.find_all('td', class_='ng-star-inserted')
        row = []
        for cell in row_cell:
            row.append(cell.text.strip())
        rows.append(row)

    
    output = []
    output.append(main_trans_head)
    output.append(main_trans_cont)
    
    output.append(headers)

    for row in main_temp:
        output.append(row)
    output.append([])
    return output

In [ ]:
output = []

for row in main_trs:
    link_cell = row.find('div', class_='link')
    if link_cell:
        html_id = link_cell.get('id')
        print(html_id)
        driver.find_element_by_id(html_id).click()
        time.sleep(3)

        # Run function on each page
        output_temp = per_subpage(BeautifulSoup(driver.page_source, 'html.parser'))
        output.append(output_temp)
        
        driver.back()
        time.sleep(3)
        #break
        for row in output_temp:
            print(row)
print(f'Done with page {url}! Add a different url at top of notebook to crawl something else.')

In [ ]:
path = f'{root}{output_csv}'
with open(path, mode='w') as nets_file:
    nets_writer = csv.writer(nets_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for page in output:
        for row in page:
            nets_writer.writerow(row)